In [2]:
import os, requests
from dotenv import load_dotenv

# Carga .env para esta ejecución
load_dotenv()

HOST = (os.getenv("DATABRICKS_HOST") or "").rstrip("/")
TOKEN = os.getenv("DATABRICKS_TOKEN") or ""

ORG_ID = "4044162781254195"  # opcional; si no te funciona, quítalo

r = requests.get(
    f"{HOST}/api/2.0/clusters/spark-versions",
    headers={"Authorization": f"Bearer {TOKEN}", "X-Databricks-Org-Id": ORG_ID},
    timeout=10,
)

print("HOST:", HOST)
print("STATUS:", r.status_code, "| OK?:", r.ok)
print(r.text[:500])


HOST: https://dbc-54859cf0-9d64.cloud.databricks.com
STATUS: 200 | OK?: True
{"versions":[{"key":"16.3.x-photon-scala2.12","name":"16.3 Photon (includes Apache Spark 3.5.2, Scala 2.12)"},{"key":"12.2.x-scala2.12","name":"12.2 LTS (includes Apache Spark 3.3.2, Scala 2.12)"},{"key":"11.3.x-photon-scala2.12","name":"11.3 LTS Photon (includes Apache Spark 3.3.0, Scala 2.12)"},{"key":"16.4.x-cpu-ml-scala2.12","name":"16.4 LTS ML (includes Apache Spark 3.5.2, Scala 2.12)"},{"key":"17.2.x-scala2.13","name":"17.2 Beta (includes Apache Spark 4.0.0, Scala 2.13)"},{"key":"17.1.x-cp
